# Working with 3rd party (detached) Prompts/Prompt Template Assets using OpenAI



This notebook should be run using with Runtime 22.2 & Python 3.10 or greater runtime environment, if you are viewing this in Watson Studio, and do not see Python 3.10.x in the upper right corner of your screen, please update the runtime now. 

## Prerequisite


* Requires a CSV file containing the test data that needs to be evaluated
* Requires the ID of project in which you want to create the prompt template asset
* Requires the ID of the deployment space in which you want to deploy the prompt template asset
* Requires COS information to store the prompt template asset

### Contents

- [Setup](#settingup)
- [Create Prompt template](#prompt)
- [Prompt Setup](#ptatsetup)



## Setup <a name="settingup"></a>

In [1]:
!pip install --upgrade datasets==2.10.0 --no-cache | tail -n 1
!pip install --upgrade evaluate --no-cache | tail -n 1
!pip install --upgrade ibm-aigov-facts-client | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1
!pip install --upgrade matplotlib | tail -n 1
!pip install --upgrade ibm_watsonx_ai | tail -n 1
!pip install --upgrade pydantic==1.10.11 --no-cache | tail -n 1
!pip install --upgrade sacrebleu --no-cache | tail -n 1
!pip install --upgrade sacremoses --no-cache | tail -n 1
!pip install --upgrade textstat --no-cache | tail -n 1
!pip install --upgrade transformers --no-cache | tail -n 1
!pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.

### Provision services and configure credentials

INSERT ALL DETAILS COLLECTED FROM THE UI STEPS INTO THESE FIELDS

In [16]:
CLOUD_API_KEY = "INSERT HERE" # INSERT YOUR_CLOUD_API_KEY
PROJECT_ID = "INSERT HERE" # INSERT PROJECT ID
existing_space_id = "INSERT HERE" # INSERT DEPLOYMENT ID
COS_RESOURCE_CRN = "INSERT HERE" # INSERT COS ID
project_access_token = "INSERT HERE" # INSERT PROJECT TOKEN
openai_api = "INSERT HERE" # OPENAI API

In [17]:
use_cpd = False

if use_cpd:
    CPD_URL = "<EDIT THIS>"
    CPD_USERNAME = "<EDIT THIS>"
    CPD_PASSWORD = "<EDIT THIS>"
    WOS_SERVICE_INSTANCE_ID = "<EDIT THIS>" # If None, default instance would be used
else:
    IAM_URL = "https://iam.cloud.ibm.com"
    DATAPLATFORM_URL = "https://api.dataplatform.cloud.ibm.com"
    SERVICE_URL = "https://aiopenscale.cloud.ibm.com"
    CLOUD_API_KEY = CLOUD_API_KEY # YOUR_CLOUD_API_KEY
    WML_CREDENTIALS = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey": CLOUD_API_KEY
    }

## Read space id from user

User can use an existing space or can create a new space to promote the model. User should choose any of these options with the below variable.

In [18]:
use_existing_space = True # Set it as False if user wants to create a new space

In [19]:
import json
from ibm_watsonx_ai import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.3.8'

### Below details are required only if user choose to use an existing space

In order to use an existing space, User can directly add the space id in the below cell

In [20]:
wml_client.spaces.list()

,ID,NAME,CREATED
0,f3d4f9dd-6516-479e-92b5-716f3f945463,SimonSV - Deployment,2025-04-16T09:16:12.862Z


### Setting up the space

In [21]:
if use_existing_space == True:
    space_id = existing_space_id
else:
    space_meta_data = {
        wml_client.spaces.ConfigurationMetaNames.NAME: space_name,
        wml_client.spaces.ConfigurationMetaNames.STORAGE: {"resource_crn":COS_RESOURCE_CRN},
        wml_client.spaces.ConfigurationMetaNames.COMPUTE: {"name": WML_INSTANCE_NAME, "crn": WML_CRN},
        wml_client.spaces.ConfigurationMetaNames.TYPE: "wx"
    }
    space_id = wml_client.spaces.store(
        meta_props=space_meta_data)["metadata"]["id"]
wml_client.set.default_space(space_id)

'SUCCESS'

## Function to create the access token

This function generates an IAM access token using the provided credentials. The API calls for creating and scoring prompt template assets utilize the token generated by this function.

In [22]:
import requests, json
def generate_access_token():
    headers={}
    if not use_cpd: 
        headers["Content-Type"] = "application/x-www-form-urlencoded"
        headers["Accept"] = "application/json"
        data = {
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": CLOUD_API_KEY,
            "response_type": "cloud_iam"
        }
        response = requests.post(IAM_URL + "/identity/token", data=data, headers=headers)
        json_data = response.json()
        iam_access_token = json_data["access_token"]
    else:
        headers["Content-Type"] = "application/json"
        headers["Accept"] = "application/json"
        data = {
            "username": CPD_USERNAME,
            "password": CPD_PASSWORD
        }
        data = json.dumps(data).encode("utf-8")
        url = CPD_URL + "/icp4d-api/v1/authorize"
        response = requests.post(url=url, data=data, headers=headers, verify=False)
        json_data = response.json()
        iam_access_token = json_data["token"]
        
    return iam_access_token

iam_access_token = generate_access_token()

## Test data containing the summarization output from model and the reference data

In [23]:
!rm -fr llm_content.csv
!wget "https://raw.githubusercontent.com/cwong79/Evaluate-and-Monitor-your-3rd-Party-AI-Models/refs/heads/main/summarization.csv"

--2025-04-16 15:33:39--  https://raw.githubusercontent.com/cwong79/Evaluate-and-Monitor-your-3rd-Party-AI-Models/refs/heads/main/summarization.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14173 (14K) [text/plain]
Saving to: ‘summarization.csv.2’

summarization.csv.2 100%[===================>]  13.84K  --.-KB/s    in 0s      

2025-04-16 15:33:40 (49.2 MB/s) - ‘summarization.csv.2’ saved [14173/14173]



In [26]:
import pandas as pd
import numpy as np
llm_data_all = pd.read_csv("summarization.csv",encoding='latin-1')

In [27]:
llm_data = llm_data_all.head(10)

# Azure OpenAI Model Invocation

In [28]:
import os
import openai
from openai import OpenAI

client = OpenAI(
  api_key=openai_api
)

## Get the Azure OpenAI deployment details from Azure OpenAI Studio

In [35]:
llm_data.head()

,original_text,generated_text,ground_truth
0,Scientists have discovered a new species of de...,New bioluminescent fish species found in deep ...,Discovery of deep-sea fish emitting soothing l...
1,An international team of astronomers has ident...,Distant exoplanet\'s water vapor-filled atmosp...,Astronomers identify exoplanet with water vapo...
2,Researchers have developed a novel nanotechnol...,New nanotechnology-based cancer treatment demo...,Researchers create cancer treatment using nano...
3,A new app is aiming to reduce food waste by co...,App connects local restaurants with customers ...,New sustainability-focused app facilitates sal...
4,Archaeologists have uncovered an ancient city ...,"Ancient city dating back over 4,000 years disc...",Archaeological find in Iraq reveals ancient ci...


### Sample generated output

In [36]:
# Function to generate summary
def generate_summary(text):
    response = client.chat.completions.create(
        model="gpt-4o",  # Use your deployment name here
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Summarize the following text: {text}"}
        ],
        max_tokens=150,  # Adjust the token limit as needed
        temperature=0.5,  # Adjust for creativity
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content.strip()

In [37]:
llm_data.loc[:, 'generated_text'] = llm_data['original_text'].apply(generate_summary) #appending generated summary

In [38]:
llm_data

,original_text,generated_text,ground_truth
0,Scientists have discovered a new species of de...,Scientists have found a new species of deep-se...,Discovery of deep-sea fish emitting soothing l...
1,An international team of astronomers has ident...,An international team of astronomers has disco...,Astronomers identify exoplanet with water vapo...
2,Researchers have developed a novel nanotechnol...,Researchers have created a new nanotechnology-...,Researchers create cancer treatment using nano...
3,A new app is aiming to reduce food waste by co...,A new app seeks to reduce food waste by linkin...,New sustainability-focused app facilitates sal...
4,Archaeologists have uncovered an ancient city ...,Archaeologists have discovered an ancient city...,Archaeological find in Iraq reveals ancient ci...
5,Researchers have developed a new type of solar...,Researchers have created a new solar panel tha...,New solar panels could revolutionize renewable...
6,"According to a recent study, regular exercise ...",A recent study indicates that regular exercise...,New study suggests that staying physically act...
7,Scientists have developed a new type of batter...,Scientists have created a new battery that can...,Battery technology innovation enables wireless...
8,A team of engineers has designed a lightweight...,Engineers have created a lightweight exoskelet...,Newly designed exoskeleton offers mobility ass...
9,NASA\'s latest space telescope has captured st...,NASA's new space telescope has taken remarkabl...,Stunning images of distant galaxies captured b...


# Create Prompt template <a name="prompt"></a>

In [39]:
from project_lib import Project
project = Project(project_id=PROJECT_ID, project_access_token=project_access_token)
pc = project.project_context

project.save_data(file_name = "summarization.csv", data = llm_data.to_csv(index=False))

project_lib is deprecated and will no longer be available in IBM Runtime 25.1.
Please upgrade to ibm_watson_studio_lib instead.


{'file_name': 'summarization.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'simonsvsummarization-donotdelete-pr-s5qcfmxihhhhdr',
 'asset_id': '0b3fcc40-b421-4d6d-80d2-c647ee7f2231'}

Create a prompt template for a summarization task

In [40]:
from ibm_aigov_facts_client import AIGovFactsClient

if not use_cpd:
    facts_client = AIGovFactsClient(
        api_key=CLOUD_API_KEY,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )
else:
    from ibm_aigov_facts_client import CloudPakforDataConfig
    creds=CloudPakforDataConfig(
        service_url=CPD_URL,
        username=CPD_USERNAME,
        password=CPD_PASSWORD
    )
    facts_client = AIGovFactsClient(
        cloud_pak_for_data_configs=creds,
        container_id=PROJECT_ID,
        container_type="project",
        disable_tracing=True
    )

In [41]:
from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate

detached_information = DetachedPromptTemplate(
    prompt_id="detached_prompt",
    model_id="proj_jl2dcegko4SnijQAIVMeahGQ",
    model_provider="OpenAI",
    model_name="openai/gpt-4o",
    model_url="https://openai.com/index/hello-gpt-4o/",
    prompt_url="https://github.com/marketplace/models/azure-openai/gpt-4o?tab=readme",
    prompt_additional_info={"model_owner": "openai"},
)

task_id = "summarization"
name = "External prompt sample (openai)"
description = "GPT-4o is a multilingual, multimodal generative pre-trained transformer developed by OpenAI and released in May 2024"
model_id = "gpt-4o"


# define parameters for PromptTemplate
prompt_variables = {"original_text": ""}
input = "{original_text}"
input_prefix= "Input:You are a helpful assistant, summarize the following text"
output_prefix= "Output:gpt-4o model summarization"

prompt_template = PromptTemplate(
    input=input,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix
)

pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

2025/04/16 15:36:20 INFO : ------------------------------ Detached Prompt Creation Started ------------------------------
2025/04/16 15:36:21 INFO : The detached prompt with ID 993e1c6e-89f1-4923-8c6a-f4f103549178 was created successfully in container_id 67573f2a-96e4-4f78-9e8a-d2a9ed1ba207.


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now navigate to the prompt template asset in your project / spaceand click on the Evaluate tab to visualise the results on the UI.